In [5]:
import pandas as pd
import numpy as np
import math
df = pd.read_csv("RealTestSample.csv")
df["Usable"] = True

def RSI(df, period=14):

    for i in range(len(df)):
        if i < period:
            df.loc[i, 'Usable'] = False
            df.loc[i, 'RSI'] = -1.0
        else:
            avgGain = 0
            avgLoss = 0
            for j in range(i - period + 1, i + 1):
                price_change = df.loc[j, 'Close'] - df.loc[j - 1, 'Close']
                if price_change > 0:
                    avgGain += price_change
                elif price_change < 0:
                    avgLoss += abs(price_change)  # Use abs to ensure positive loss values

            avgGain /= period
            avgLoss /= period
            avgLoss = max(avgLoss, 0.001)  # Ensure avgLoss is not zero to avoid division by zero

            RS = avgGain / avgLoss
            df.loc[i, 'RSI'] = 100 - (100 / (1 + RS))

    return df

df = RSI(df, 14)

def EMA(df, column, period):
    smoothingFactor = 2 / (period + 1)
    df['temp'] = 0.0
    for i in range(len(df)):
        if i < period:
            df.loc[i, 'Usable'] = False
            df.loc[i, 'EMA'] = -1.0
        else:
            for j in range(i - period + 1, i):
                if j == i - period:
                    df.loc[j, 'temp'] = df.loc[j, column]
                else:
                    df.loc[j, 'temp'] = (df.loc[j, column] * smoothingFactor) + (df.loc[j-1, 'temp'] * (1 - smoothingFactor))
            df.loc[i, 'EMA'] = (df.loc[i, column] * smoothingFactor) + (df.loc[i-1, 'temp'] * (1 - smoothingFactor))

    
    df = df.drop("temp", axis=1)
    df['EvC'] = df['EMA'] / df[column]
    return df

df = EMA(df, 'Close', 10)

def SMA (df, period):
    for i in range(len(df)):
        if i < period:
            df.loc[i, 'Usable'] = False
            df.loc[i, 'SMA'] = -1.0
        else:
            sum = 0.0
            for j in range(i - period + 1, i + 1):
                sum += df.loc[j, 'Close']
            df.loc[i, 'SMA'] = sum / period
    return df

def normalize(df, column1, column2):
    df['normalized_' + column1 + '_' + column2] = abs(df[column1] - df[column2]) / (df[column1] + df[column2])
    return df

df = SMA(df, 14)
df = normalize(df, 'SMA', 'EMA')

def MACD(df, fast_period, slow_period, signal_period):
    # Ensure the original DataFrame is not altered
    temp_df = df[['Close']].copy()
    
    # Calculate fast and slow EMAs
    ema_fast = EMA(temp_df.copy(), 'Close', fast_period)
    ema_slow = EMA(temp_df.copy(), 'Close', slow_period)
    
    # Calculate the MACD line
    df['MACDLine'] = ema_fast['EMA'] - ema_slow['EMA']
    
    # Calculate the Signal line using the correct column name
    signal_input = pd.DataFrame(df['MACDLine']).rename(columns={'MACDLine': 'Close'})  # Rename for compatibility with EMA function
    df['MACDSig'] = EMA(signal_input, 'Close', signal_period)['EMA']
    
    # Set 'Usable' flag based on the maximum of the periods used
    max_period = max(fast_period, slow_period, signal_period)
    df.loc[:max_period-1, 'Usable'] = False
    
    return df

# Example usage
df = MACD(df, 12, 26, 9)

def BollingerBands(df, period, stdCount):
    # Calculate the SMA (Simple Moving Average)
    df['SMA'] = df['Close'].rolling(window=period).mean()
    
    # Initialize UpperBand and LowerBand columns
    df['UpperBand'] = 0.0
    df['LowerBand'] = 0.0

    for i in range(len(df)):
        if i < period:
            df.loc[i, 'Usable'] = False
        else:
            ss = 0
            # Calculate the sum of squared differences from SMA
            for j in range(i - period + 1, i + 1):
                ss += (df.loc[j, 'Close'] - df.loc[i, 'SMA']) ** 2
            
            # Correct the formula for standard deviation
            std = math.sqrt(ss / (period - 1))
            
            # Calculate the Upper and Lower Bands
            df.loc[i, 'UpperBand'] = df.loc[i, 'SMA'] + (std * stdCount)
            df.loc[i, 'LowerBand'] = df.loc[i, 'SMA'] - (std * stdCount)
            df.loc[i, 'STD'] = std
    return df

df = BollingerBands(df, 9, 2)
df = normalize(df, 'SMA', 'STD')
df['TypicalPrice'] = (df['Close'] + df['High'] + df['Low']) / 3

def VWAPperiod(df, period):
    for i in range(len(df)):
        if i < period:
            df.loc[i, 'Usable'] = False
            df.loc[i, 'VWAP'] = -1.000  # Or another placeholder for non-usable data
        else:
            volumeSum = 0
            totalSum = 0
            for j in range(i - period + 1, i + 1):  # Loop through the period
                volumeSum += df.loc[j, 'Volume']  # Use j instead of i
                totalSum += df.loc[j, 'Volume'] * df.loc[j, 'TypicalPrice']  # Use j here as well
            df.loc[i, 'VWAP'] = totalSum / volumeSum  # Calculate VWAP after summing for the entire period
    return df

df = VWAPperiod(df, 5)

def highestHigh(df, period, column, i):
    highestVal = 0
    for j in range (i - period + 1, i + 1):
        if df.loc[j, column] > highestVal:
            highestVal = df.loc[j, column]
    return highestVal
    
def lowestLow(df, period, column, i):
    lowestVal = 9999999999
    for j in range (i - period + 1, i + 1):
        if df.loc[j, column] < lowestVal:
            lowestVal = df.loc[j, column]
    return lowestVal

def ichimokucloud(df, period1, period2, period3):
    for i in range(len(df)):
        if i < max(period1, period2, period3):
            df.loc[i, 'Usable'] = False
            df.loc[i, 'Tenkan'] = -1.0
            df.loc[i, 'Kijun'] = -1.0
            df.loc[i, 'SenkouA'] = -1.0
            df.loc[i, 'SenkouB'] = -1.0
            df.loc[i, 'Chikou'] = -1.0
        else:
            df.loc[i, 'Tenkan'] = (highestHigh(df, period1, 'High', i) + lowestLow(df, period1, 'Low', i)) / 2
            df.loc[i, 'Kijun'] = (highestHigh(df, period2, 'High', i) + lowestLow(df, period2, 'Low', i)) / 2
            df.loc[i, 'SenkouA'] = (df.loc[i, 'Tenkan'] + df.loc[i, 'Kijun']) / 2
            df.loc[i, 'SenkouB'] = (highestHigh(df, period3, 'High', i) + lowestLow(df, period3, 'Low', i)) / 2
            df.loc[i, 'Chikou'] = df.loc[i - period2, 'Close']
    return df

df = ichimokucloud(df, 9, 26, 52)

for i in range(len(df)):
    if i == 0:
        df.loc[i, 'TR'] = df.loc[i, 'High'] - df.loc[i, 'Low']
    else:
        df.loc[i, 'TR'] = max(df.loc[i, 'High'] - df.loc[i, 'Low'], abs(df.loc[i, 'High'] - df.loc[i - 1, 'Close']), abs(df.loc[i, 'Low'] - df.loc[i - 1, 'Close']))

def ATR(df, period):
    for i in range(len(df)):
        if i < period:
            df.loc[i, 'Usable'] = False
            df.loc[i, 'ATR'] = -1.0
        else:
            ATR = 0
            for j in range(i - period + 1, i + 1):
                ATR += df.loc[j, 'TR']
            df.loc[i, 'ATR'] = ATR / period
    return df

df = ATR(df, 9)
df = normalize(df, 'ATR', 'TR')
df = normalize(df, 'ATR', 'SMA')
def gtr(df):
    for i in range(len(df)):
        if i == len(df) - 1:
            df.loc[i, 'INC'] = int(0)
        elif df.loc[i, 'Close'] < df.loc[i + 1, 'Close']:
            df.loc[i, 'INC'] = int(1)
        else:
            df.loc[i, 'INC'] = int(0)
    return df

df = gtr(df)

df = normalize(df, 'TypicalPrice', 'ATR')

def swing(df, period):
    for i in range(len(df)):
        if i < period:
            df.loc[i, 'Usable'] = False
            df.loc[i, 'Swing'] = 0
        else:
            if df.loc[i, 'Close'] > df.loc[i - period, 'Close'] * 1.1:
                df.loc[i, 'Swing'] = 1.0
            elif df.loc[i, 'Close'] * 1.1 < df.loc[i- period, 'Close']:
                df.loc[i, 'Swing'] = -1.0
            elif df.loc[i, 'Close'] * 1.05 < df.loc[i- period, 'Close']:
                df.loc[i, 'Swing'] = -0.5
            elif df.loc[i, 'Close'] > df.loc[i - period, 'Close'] * 1.1:
                df.loc[i, 'Swing'] = 0.5
            else:
                df.loc[i, 'Swing'] = 0
    return df

df = swing(df, 7)

def onbalancevolume(df):
    for i in range(len(df)):
        if i == 0:
            df.loc[i, 'OBV'] = 0
        else:
            if df.loc[i, 'Close'] > df.loc[i-1, 'Close']:
                df.loc[i, 'OBV'] = df.loc[i-1, 'OBV'] + df.loc[i, 'Volume']
            else:
                df.loc[i, 'OBV'] = df.loc[i-1, 'OBV'] - df.loc[i, 'Volume']
    
    return df

df = onbalancevolume(df)

def custom_log10(value):
    if value > 0:
        return np.log10(value)
    elif value < 0:
        return -np.log10(abs(value))
    else:
        return np.nan  # Handle the case where value is 0 (log10(0) is undefined)

# Apply the function to the 'OBV' column
df['OBV'] = df['OBV'].apply(custom_log10)

def periodicObv(df, period):
    df['temp'] = 0
    for i in range(len(df)):
        if i < period:
            df.loc[i, 'POBV'] = -1  # Mark as unusable for the first 'period' rows
            df.loc[i, 'Usable'] = False
        else:
            df.loc[i, 'temp'] = 0  # Reset temp for current row
            for j in range(i - period + 1, i + 1):  # Look back 'period' rows
                if df.loc[j, 'Close'] > df.loc[j - 1, 'Close']:
                    df.loc[i, 'temp'] += df.loc[j, 'Volume']  # Add volume if close is higher
                elif df.loc[j, 'Close'] < df.loc[j - 1, 'Close']:
                    df.loc[i, 'temp'] -= df.loc[j, 'Volume']  # Subtract volume if close is lower
            df.loc[i, 'POBV'] = df.loc[i, 'temp']  # Assign the temp value to POBV column
    
    df = df.drop(columns=['temp'])  # Clean up
    df['POBV'] = df['POBV'].apply(custom_log10)
    return df

df = periodicObv(df, 8)

df['VWAPvTP'] = (df['VWAP'] - df['TypicalPrice']) / (df['VWAP'] + df['TypicalPrice'] + 0.000001) * 100

def StochasticOscillator(df, period1, period2):
    kline_values = []
    for i in range(period1 - 1, len(df)):
        low = lowestLow(df, period1, 'Low', i)
        high = highestHigh(df, period1, 'High', i)
        k_value = (df.loc[i, 'Close'] - low) / (high - low) * 100
        kline_values.append(k_value)

    # Add NaN for the first 'period1 - 1' rows where Stochastic Oscillator is not calculated
    df['KLine'] = [None] * (period1 - 1) + kline_values
    df['DLine'] = df['KLine'].rolling(window=period2).mean()
    df['KvD'] = (df['KLine'] - df['DLine']) / (df['KLine'] + df['DLine'] + 0.00001) * 100
    return df
    
df = StochasticOscillator(df, 14, 3)

def calculate_CCI(df, period):
    # Step 2: Calculate the SMA of the 'TypicalPrice'
    df['SMA(TP)'] = df['TypicalPrice'].rolling(window=period).mean()

    # Step 3: Calculate the Mean Absolute Deviation (MAD) manually
    df['MAD'] = df['TypicalPrice'].rolling(window=period).apply(lambda x: (abs(x - x.mean())).mean(), raw=False)

    # Step 4: Calculate CCI
    df['CCI'] = (df['TypicalPrice'] - df['SMA(TP)']) / (0.015 * df['MAD'])

    return df

df = calculate_CCI(df, 5)


def calculate_relative_volume(df, period):
    """
    Calculate the Relative Volume (RVOL) for each period.

    :param df: DataFrame that contains at least a 'Volume' column.
    :param period: The period over which to calculate the average volume (e.g., 10 days).
    :return: DataFrame with an added 'RelativeVolume' column.
    """
    # Ensure we are working on a copy of the DataFrame to avoid SettingWithCopyWarning
    df = df.copy()

    # Calculate the rolling average volume over the specified period
    df.loc[:, 'AverageVolume'] = df['Volume'].rolling(window=period).mean()

    # Calculate the Relative Volume (RVOL)
    df.loc[:, 'RelativeVolume'] = df['Volume'] / df['AverageVolume']

    return df

df = calculate_relative_volume(df, 3)


def WilliamsR(df, period):
    for i in range(len(df)):
        if i < period:
            df.loc[i, 'Usable'] = False
            df.loc[i, 'Williams'] = 0.0
        else:
            df.loc[i, 'Williams'] = (highestHigh(df, period, 'High', i) - df.loc[i, 'Close']) / (highestHigh(df, period, 'High', i) - lowestLow(df, period, 'Low', i)) * -100
    return df

df = WilliamsR(df, 14)

def calculate_parabolic_sar(df, af_start, af_step, af_max):
    """
    Calculate the Parabolic SAR for a DataFrame with 'High' and 'Low' columns.

    :param df: DataFrame containing 'High' and 'Low' prices.
    :param af_start: Starting acceleration factor (default 0.02).
    :param af_step: Step increment for the acceleration factor (default 0.02).
    :param af_max: Maximum acceleration factor (default 0.2).
    :return: DataFrame with an added 'ParabolicSAR' column.
    """
    # Initialize SAR-related variables
    sar = df['Low'][0]  # Initial SAR value (for uptrend)
    ep = df['High'][0]  # Initial extreme point (for uptrend)
    af = af_start  # Initial acceleration factor
    trend = 1  # 1 for uptrend, -1 for downtrend

    sar_values = []

    for i in range(1, len(df)):
        # Calculate SAR based on current trend direction
        if trend == 1:  # Uptrend
            sar_new = sar + af * (ep - sar)
            if df['Low'][i] < sar_new:  # Trend reversal
                trend = -1
                sar_new = ep
                af = af_start
                ep = df['Low'][i]
            else:
                if df['High'][i] > ep:  # New extreme point
                    ep = df['High'][i]
                    af = min(af + af_step, af_max)
        else:  # Downtrend
            sar_new = sar - af * (sar - ep)
            if df['High'][i] > sar_new:  # Trend reversal
                trend = 1
                sar_new = ep
                af = af_start
                ep = df['High'][i]
            else:
                if df['Low'][i] < ep:  # New extreme point
                    ep = df['Low'][i]
                    af = min(af + af_step, af_max)

        sar_values.append(sar_new)
        sar = sar_new

    # Add the SAR values to the DataFrame
    df['ParabolicSAR'] = [None] + sar_values  # Adding None for the first period as SAR is undefined

    return df

df = calculate_parabolic_sar(df, 0.02, 0.02, 0.2)

df = normalize(df, 'ParabolicSAR', 'TypicalPrice')

def calculate_adl_and_chaikin(df, period):
    """
    Calculate the Accumulation/Distribution Line (ADL) for each period.

    :param df: DataFrame containing 'High', 'Low', 'Close', and 'Volume' columns.
    :return: DataFrame with an added 'ADL' column.
    """
    # Step 1: Calculate Money Flow Multiplier (MFM)
    df['MFM'] = ((df['Close'] - df['Low']) - (df['High'] - df['Close'])) / (df['High'] - df['Low'])

    # Step 2: Calculate Money Flow Volume (MFV)
    df['MFV'] = df['MFM'] * df['Volume']

    # Step 3: Calculate Accumulation/Distribution Line (ADL)
    df['ADL'] = df['MFV'].cumsum()

    df['CMF'] = df['MFV'].rolling(window=period).sum() / df['Volume'].rolling(window=period).sum()

    return df

df = calculate_adl_and_chaikin(df, 20)


def add_roc_columns(df):
    """
    Adds two columns to the DataFrame:
    - dayROC: Rate of Change 1 period ago.
    - longROC: Rate of Change 10 periods ago.
    
    The default periods are 1 (dayROC) and 10 (longROC).
    """
    # Calculate dayROC (Rate of Change for 1 period ago)
    df['dayROC'] = (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1) * 100
    
    # Calculate longROC (Rate of Change for 10 periods ago)
    df['longROC'] = (df['Close'] - df['Close'].shift(10)) / df['Close'].shift(10) * 100
    
    return df

add_roc_columns(df)

def donchians(df, period):
    df['dUp'] = df['High'].rolling(window=period).max()
    
    # Calculate Lower Channel (Lowest Low over the period)
    df['dLow'] = df['Low'].rolling(window=period).min()
    
    # Calculate Middle Channel (Average of Upper and Lower Channels)
    df['dMid'] = (df['dUp'] + df['dLow']) / 2
    
    return df

donchians(df, 7)

def calculate_heikin_ashi(df):
    """
    Calculate Heikin Ashi candles for a given DataFrame.
    
    :param df: DataFrame containing 'Open', 'High', 'Low', 'Close' columns.
    :return: DataFrame with added Heikin Ashi 'HA_Open', 'HA_High', 'HA_Low', 'HA_Close' columns.
    """
    # Step 1: Calculate Heikin Ashi Close (average of Open, High, Low, Close)
    df['HA_Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    
    # Step 2: Initialize Heikin Ashi Open as a copy of the original Open
    df['HA_Open'] = df['Open']
    
    # Step 3: Calculate Heikin Ashi Open (average of previous HA_Open and previous HA_Close)
    for i in range(1, len(df)):
        df.loc[i, 'HA_Open'] = (df.loc[i-1, 'HA_Open'] + df.loc[i-1, 'HA_Close']) / 2
    
    # Step 4: Calculate Heikin Ashi High (max of current High, HA_Open, and HA_Close)
    df['HA_High'] = df[['High', 'HA_Open', 'HA_Close']].max(axis=1)
    
    # Step 5: Calculate Heikin Ashi Low (min of current Low, HA_Open, and HA_Close)
    df['HA_Low'] = df[['Low', 'HA_Open', 'HA_Close']].min(axis=1)
    
    return df

def compare_heikin_ashi_vs_normal(df):
    """
    Compare Heikin Ashi OHLC with normal OHLC and create difference variables.
    """
    # Step 1: Calculate Heikin Ashi OHLC
    df = calculate_heikin_ashi(df)
    
    # Step 2: Compare Heikin Ashi vs Normal Candles (difference between OHLC)
    df['Open_Diff'] = df['HA_Open'] - df['Open']
    df['Close_Diff'] = df['HA_Close'] - df['Close']
    df['High_Diff'] = df['HA_High'] - df['High']
    df['Low_Diff'] = df['HA_Low'] - df['Low']
    
    # Step 3: Compare candle body size (difference between open and close for Heikin Ashi vs normal candles)
    df['Normal_Candle_Body'] = df['Close'] - df['Open']
    df['HA_Candle_Body'] = df['HA_Close'] - df['HA_Open']

    df['Body_Diff'] = df['Normal_Candle_Body'] - df['HA_Candle_Body']
    df['Normalized_Body_Diff'] = df['Body_Diff'] / df['Normal_Candle_Body'].replace(0, 1)
    
    return df

compare_heikin_ashi_vs_normal(df)

def calculate_vortex_indicator(df, period=14):
    """
    Calculate the Vortex Indicator (VI+ and VI-).
    
    :param df: DataFrame with 'High', 'Low', and 'Close' columns.
    :param period: The period for the Vortex calculation (default is 14).
    :return: DataFrame with added 'VI+' and 'VI-' columns.
    """
    df['VM+'] = abs(df['High'] - df['Low'].shift(1))
    df['VM-'] = abs(df['Low'] - df['High'].shift(1))
    
    # Step 3: Rolling sum of TR, VM+, and VM-
    df['TR_sum'] = df['TR'].rolling(window=period).sum()
    df['VM+_sum'] = df['VM+'].rolling(window=period).sum()
    df['VM-_sum'] = df['VM-'].rolling(window=period).sum()
    
    # Step 4: Calculate the Vortex Indicator (VI+ and VI-)
    df['VI+'] = df['VM+_sum'] / df['TR_sum']
    df['VI-'] = df['VM-_sum'] / df['TR_sum']
    
    return df

# Example usage
df = calculate_vortex_indicator(df)

def calculate_aroon_indicator(df, period=25):
    """
    Calculate the Aroon Indicator (Aroon Up and Aroon Down).
    
    :param df: DataFrame with 'High' and 'Low' columns.
    :param period: The period for the Aroon calculation (default is 25).
    :return: DataFrame with added 'Aroon Up' and 'Aroon Down' columns.
    """
    df['Aroon Up'] = df['High'].rolling(window=period).apply(lambda x: ((period - (period - 1 - x.argmax())) / period) * 100, raw=True)
    df['Aroon Down'] = df['Low'].rolling(window=period).apply(lambda x: ((period - (period - 1 - x.argmin())) / period) * 100, raw=True)
    
    return df

# Example usage
df = calculate_aroon_indicator(df)

def calculate_cmo(df, period=14):
    """
    Calculate the Chande Momentum Oscillator (CMO) and add it directly to the DataFrame.
    
    :param df: DataFrame with a 'Close' price column.
    :param period: The period for the CMO calculation (default is 14).
    :return: DataFrame with an added 'CMO' column.
    """
    # Initialize a list to store the CMO values
    cmo_values = []
    
    # Iterate over the DataFrame starting from the 'period' index
    for i in range(len(df)):
        if i < period:
            # Append NaN for the first 'period' entries since we can't calculate CMO
            cmo_values.append(None)
        else:
            # Calculate the sum of up moves and down moves
            up_sum = 0
            down_sum = 0
            for j in range(i - period + 1, i + 1):
                change = df['Close'].iloc[j] - df['Close'].iloc[j - 1]
                if change > 0:
                    up_sum += change
                elif change < 0:
                    down_sum += abs(change)
            
            # Calculate the CMO using the up_sum and down_sum
            cmo = 100 * (up_sum - down_sum) / (up_sum + down_sum) if (up_sum + down_sum) != 0 else 0
            cmo_values.append(cmo)
    
    # Add the CMO column to the DataFrame
    df['CMO'] = cmo_values
    
    return df

calculate_cmo(df, 14)

def reorder_columns(df):
    # List of the first columns in the desired order
    first_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

    # Get the rest of the columns
    remaining_columns = [col for col in df.columns if col not in first_columns]

    # Sort the remaining columns alphabetically
    remaining_columns.sort()

    # Reorder the DataFrame
    df = df[first_columns + remaining_columns]

    return df

df = reorder_columns(df)

In [12]:
df = df.drop(columns=['Unnamed: 0'])

In [13]:
df.to_csv("ETHNEWTEST.csv")